In [1]:
import pandas as pd
import numpy as np
from tqdm import *

In [2]:
data_coach = pd.read_csv("data/coaches.csv")
data_athletes = pd.read_csv("data/athletes.csv")
data_programs = pd.read_csv("data/programs.csv")
years_index = range(1896, 2025, 4)
programs_index = pd.read_csv('data/programs_index.csv')['program_id'].tolist()

In [3]:
years_befor_start = 2 * 4
years_after_end = 2 * 4
coath_time_loss = 0.2

In [4]:
def make_dataset(noc, team_now, team_sum, sex="A"):

    tmp_athletes = data_athletes[data_athletes["NOC"] == noc]
    result_df = pd.DataFrame()
    if tmp_athletes.shape[0] == 0:
        return result_df
    
    num_participate = 0

    # each year
    tqdm_bar = tqdm(years_index)

    for year in tqdm_bar:
        tmp_one_year = tmp_athletes[tmp_athletes["Year"] == year]
        
        if tmp_one_year.shape[0] == 0:
            continue
        
        host = tmp_one_year["Host"].iloc[0]

        tmp_df = pd.DataFrame()
        tmp_df["Year"] = [year]
        tmp_df["NOC"] = [noc]
        tmp_df["Host"] = [host]

        # each program
        for program in programs_index:
            tmp_one_year_program = tmp_one_year[tmp_one_year["Sport"] == program]
            if tmp_one_year_program.shape[0] == 0:
                tmp_df[program] = [0]
                continue
            tmp_one_year_program_athletes = list(set(tmp_one_year_program["Name"].tolist()))
            
            team_score_in_program = 0
            athletes_num_in_program = 0

            for each_athletes in tmp_one_year_program_athletes:
                
                if sex != "A":
                    tmp_athletes_record = tmp_athletes[(tmp_athletes["Name"] == each_athletes) & (tmp_athletes["Year"] < year) & (tmp_athletes["Sex"] == sex)]
                else:
                    tmp_athletes_record = tmp_athletes[(tmp_athletes["Name"] == each_athletes) & (tmp_athletes["Year"] < year)]

                if tmp_athletes_record.shape[0] == 0:
                    athletes_num_in_program += 1
                    continue
                else:
                    participate_year_list = (list(set(tmp_athletes_record["Year"])))
                    tmp_athletes_score = 0
                    for each_year in participate_year_list:
                        tmp_athletes_score += tmp_athletes_record[tmp_athletes_record["Year"] == each_year]["Medal"].mean()

                    team_score_in_program += tmp_athletes_score / len(participate_year_list)
                    athletes_num_in_program += 1

            all_program_event = (data_programs[data_programs["Code"] == program])[str(year)].iloc[0]
            tmp_df[program] = [team_score_in_program]

        result_df = pd.concat([result_df, tmp_df], ignore_index=True)

        if result_df.shape[0] == 0:
            return result_df

        tqdm_bar.set_description('Team [{}/{}], NOC: {}, Year: {}, Usable:{} '.format(team_now, team_sum, noc, year, result_df.shape[0]))


    result_df.fillna(0, inplace=True)
    return result_df

In [5]:
def make_dataset_for_one_coach(noc, team_num, team_sum, program, start_year, end_year, team_sex="A"):
    data_set_aoc_df = make_dataset(noc, team_num, team_sum, team_sex)
    data_set_one_program = data_set_aoc_df[["NOC", "Year", "Host", program]].copy()
    for each in data_set_one_program.index:
        year = data_set_one_program.loc[each, "Year"]
        data_set_one_program.loc[each, "Coach"] = 1 if year >= start_year and year <= end_year else ((1 - coath_time_loss * (year - end_year) / 4) if year > end_year else 0)
        if team_sex != "A":
            data_set_one_program.loc[each, "Medal_score"] = data_athletes[(data_athletes["NOC"] == noc) & (data_athletes["Sex"] == team_sex) & (data_athletes["Sport"] == program) & (data_athletes["Year"] == year)]["Medal"].mean() 
        else:
            data_set_one_program.loc[each, "Medal_score"] = data_athletes[(data_athletes["NOC"] == noc) & (data_athletes["Sport"] == program) & (data_athletes["Year"] == year)]["Medal"].mean()

    return data_set_one_program.fillna(0)

In [6]:
def make_dataset_for_each_coach():
    coach_name_list = []
    for i, each in enumerate(data_coach.index):
        noc = data_coach.loc[each, "NOC"]
        program = data_coach.loc[each, "Program"]
        sex = data_coach.loc[each, "Sex"]
        data_set_one_coach = make_dataset_for_one_coach(noc, i + 1, len(data_coach.index), program, data_coach.loc[each, "Start"], data_coach.loc[each, "End"])
        data_set_one_coach = data_set_one_coach[(data_set_one_coach["Year"] >= data_coach.loc[each, "Start"] - years_befor_start) & (data_set_one_coach["Year"] <= data_coach.loc[each, "End"] + years_after_end)]
        coach_file_name = str(i) + "_" + data_coach.loc[each, "Name"].replace(" ", "_")
        data_set_one_coach.to_csv("data/dataset/coach/" + coach_file_name + ".csv", index=False)        
        coach_name_list.append(coach_file_name)
    
    np.savetxt("data/dataset/coach/coach_name_list.txt", coach_name_list, fmt="%s")


In [7]:
data_athletes["Medal"] = data_athletes["Medal"].replace("Gold", 3)
data_athletes["Medal"] = data_athletes["Medal"].replace("Silver", 2)
data_athletes["Medal"] = data_athletes["Medal"].replace("Bronze", 1)
data_athletes["Medal"] = data_athletes["Medal"].replace("No medal", 0)

# make_dataset_for_one_coach("CHN", "VVO", 1996, 1996, "F")
make_dataset_for_each_coach() 


C:\Users\chen1\AppData\Local\Temp\ipykernel_3672\137557406.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_athletes["Medal"] = data_athletes["Medal"].replace("No medal", 0)
Team [12/12], NOC: CAN, Year: 2024, Usable:17 : 100%|██████████| 33/33 [00:03<00:00, 10.62it/s]
